Run the following cell to install the dependencies

In [ ]:
! pip install segments-ai
! git clone https://github.com/chenfengxu714/SqueezeSegV3.git
! cd SqueezeSegV3
! pip install -r requirements.txt
! cd ..

# Quickly label point clouds by using model predictions

## 1. Upload your point clouds and label a small subset

If you have a folder of point cloud on your pc, you can simply upload them to Segments.ai through the web interface: first create a new dataset, then upload the samples.

But let's assume your data is in the cloud, and all you have is a list of image URLs. In this case, you can upload them to Segments.ai using our API or Python SDK. You need an API key for this, which can be created on your [account page](https://segments.ai/account).

In this tutorial, our goal is to label a dataset of 10 SemanticKITTI scenes. First, we will initialize the client:

In [ ]:
from segments import SegmentsClient

api_key = '2ee27d7436c6c56f9b04e1655b51234ac9d1e88e'
api_url = 'http://localhost:5000/'
client = SegmentsClient(api_key, api_url)

Next, we will create a new dataset for point cloud segmentation.

In [ ]:
dataset_name = "pcsemseg"
task_type = 'pointcloud-segmentation'

dataset = client.add_dataset(dataset_name, task_type=task_type)
dataset_identifier = dataset['owner']['username'] + "/" + dataset_name

Now, we are ready to upload the point cloud scans to the newly created dataset.

In [ ]:
import os

# Upload the samples to Segments.ai
for filename in os.listdir('data/sequences/00/velodyne'):
  with open(os.path.join('data/sequences/00/velodyne', filename), 'rb') as f:
    pcd_asset = client.upload_asset(f, filename + '.bin')

  attributes = {
    "pcd": {
        "url": pcd_asset['url']
    },
  }

  try:
    client.add_sample(dataset_identifier, filename, attributes)
  except:
    continue

Once the images are uploaded, go to the web interface, open the new dataset, and click the "Start labeling" button on the samples tab. Rather than immediately labeling the entire dataset, let's start out by labeling 2 images.

## 2. Train a segmentation model on the labeled images

After you've labeled a few point clouds, go to the releases tab of your dataset and create a new release, for example with the name "v0.1". A release is a snapshot of your dataset at a particular point in time.

Through the Python SDK, we can now initialize a SegmentsDataset from this release and visualize the labeled images. The SegmentsDataset is compatible with popular frameworks like PyTorch, Tensorflow and Keras.

In [ ]:
from segments import SegmentsDataset

# Initialize a dataset from the release file
release = client.get_release(dataset_identifier, 'v0.1')
print(release)
dataset = SegmentsDataset(release, labelset='ground-truth', filter_by='labeled')

Next, you can train a segmentation model on the labeled samples.
However, for demonstration purposes, we will simply use a pretrained [SqueezeSegV3](https://github.com/chenfengxu714/SqueezeSegV3) model.

In [ ]:
# train your model here

## 3. Generate and upload label predictions for the unlabeled images

Now that we have a trained model, we can run it on the unlabeled point clouds to generate label predictions, and upload these predictions to Segments.ai:

In [ ]:
dataset_path = './unlabeled_data'
output_path = "./output"

In [ ]:
import urllib.request
import os
from segments import SegmentsDataset


# Initialize a new dataset, this time containing only unlabeled images
dataset = SegmentsDataset(release, labelset='ground-truth', filter_by='unlabeled')

download_path = os.path.join(dataset_path, 'sequences', '00', 'velodyne')
os.makedirs(download_path, exist_ok=True)

for sample in dataset:
  # Save 
  sample_url = sample['attributes']['pcd']['url']
  urllib.request.urlretrieve(sample_url, os.path.join(download_path, sample['name']))

In [ ]:
from utils import run_model

run_model(dataset_path, output_path)

In [ ]:
from utils import get_prediction
from segments import SegmentsDataset

predictions_path = os.path.join(output_path, 'sequences', '00', 'predictions')

for sample in dataset:
    name = sample["name"][:-4]
    label_path = os.path.join(predictions_path, name + '.label')
    annotations, point_annotations = get_prediction(label_path)

    # Upload the predictions to Segments.ai
    attributes = {
        'format_version': '0.1',
        "annotations": annotations,
        "point_annotations": point_annotations
    }
    client.add_label(sample['uuid'], 'ground-truth', attributes, label_status='PRELABELED')

## 4. Verify and correct the predicted labels

Now go back to Segments.ai and click the "Start labeling" button again to continue labeling. This time, your job is quite a bit easier: instead of having to label each image from scratch, you can simply correct the few mistakes your model made!

## Next steps

As you keep iterating between model training and labeling in this manner, your model will quickly get better and better. You'll reach a point where you're mostly just verifying the model's predictions, only having to correct the occasional mistakes on hard edge cases.

Was this useful for you? Let us know! Make sure to check out the Segments.ai [documentation](https://docs.segments.ai/python-sdk) and don't hesitate to [contact us](https://segments.ai/contact) if you have any questions.